In [1]:
import math
import numpy as np

In [ ]:
def getB(n):
    L = np.exp(np.sqrt(np.log(n)*np.log(np.log(n))))
    return math.ceil(L**(1/np.sqrt(2)))

In [12]:
def polynomial(a, n):
    return a**2 - n
    
def bSmooth(b, n, factorBase):
    a0 = math.ceil(np.sqrt(n))
    ret = {}
    while len(ret) < b:
        val = polynomial(a0, n)
        retVal = polynomial(a0, n)
        factorization = {}
        for p in factorBase:
            exponent = 0
            while val % p == 0:
                exponent += 1
                val /= p
            if exponent > 0:
                factorization[p] = exponent
            if val == 1:
                break
        if val == 1:
            ret[retVal] = factorization
        a0 += 1
    return ret

In [10]:
def getFactorBase(b, n):
    # First get all primes less than or equal to b
    seive = np.array([True]*b)
    seive[0] = False
    for i in range(2, math.ceil(b)):
        j = 2*i
        while j <= b:
            seive[j-1] = False
            j += i
    
    primes = np.array([])
    for i in range(len(seive)):
        if seive[i]:
            primes = np.append(primes, i+1)
            
    # Next check if n is a quadratic residue for odd primes
    factorBase = np.array([2])
    for i in range(1, len(primes)):
        val = n**((primes[i]-1)/2) % primes[i]
        if val == 1:
            factorBase = np.append(factorBase, primes[i])
    return factorBase

In [ ]:
bSmooth(30, 91, getFactorBase(30, 91))

In [5]:
def check_smoothness(start, batch_size, n, roots):
    """
        Finds the B-smooth elements of the sequence (x^2 - n) for each x
        from 'start' to 'start + batch_size.' 
        
        It returns a dict with the corresponding x's as the key and the
        their prime factorization as the value.
    """
    # Tracks which (x^2 - n) values in the given range can be factored by primes in B
    # If result == 1 at the end, the value has been successfully factored
    result = [int((start + i)**2) - n for i in range(batch_size)]
    # Tracks the primes used to factor the (x^2 - n) values
    
    factorization = [{} for _ in range(batch_size)]
    
    
    for i in range(result[0] % 2, batch_size, 2):
        num = 0
        while result[i] % 2 == 0:
            result[i] = result[i] // 2
            num += 1
        factorization[i][2] = num
    
    for p in roots:
        if roots[p] != None:
            start_mod_p = start % p
            
            start_r0 = 0
            while (start_mod_p + start_r0) % p != roots[p][0]:
                start_r0 += 1
            start_r1 = 0
            while (start_mod_p + start_r1) % p != roots[p][1]:
                start_r1 += 1
            for i in range(start_r0, batch_size, p):
                num = 0
                while result[i] % p == 0:
                    result[i] = result[i] // p
                    num += 1
                factorization[i][p] = num
            
            for i in range(start_r1, batch_size, p):
                num = 0
                while result[i] % p == 0:
                    result[i] = result[i] // p
                    num += 1
                if p not in factorization[i]:
                    factorization[i][p] = 0
                factorization[i][p] += num
        '''
        # Unsure if this is needed, does brute force calculations for all primes that don't have roots
        else:
            for i in range(len(result)):
                while result[i] % p == 0:
                    result[i] = result[i] // p
        '''
        

    smooth_nums = {}
    for i in range(batch_size):
        if result[i] == 1:
            smooth_nums[start + i] = factorization[i]
            
    
    return smooth_nums

In [6]:
def build_matrix(factorizations, B_primes):
    """
        Builds a matrix in the necessary form for Gaussian elimination.

        Returns the matrix and labels, which are the x values that correspond
        to each row of the matrix.

        NOTE: we may need to change this to use numpy arrays depending on our
        method for Gaussian elimination.
    """
    # Builds a map with primes as keys and their index as the value for easy lookup later
    sorted_primes = sorted(list(primes.keys()))
    primes_to_index = {2: 0}
    for i in range(len(sorted_primes)):
        primes_to_index[sorted_primes[i]] = i+1
    
    labels = []
    matrix = []
    for x in factorizations:
        if len(matrix) == len(primes_to_index):
            break
        f_list = [0 for _ in range(len(primes_to_index))]
        for p in factorizations[x]:
            f_list[primes_to_index[p]] = factorizations[x][p]
        matrix.append(f_list)
        # Might want to change labels to (x^2 - n) values later
        labels.append(x)
        
    return matrix, labels

In [ ]:
### TEST FIELD ###

n = 16921456439215439701

# Choosing B based on the article, can be changed
# Seems like C can be any value [0.5, 1]
C = 0.5
X = 2*n**(0.75)
B = math.ceil(np.exp(C*math.sqrt(math.log2(X)*math.log2(math.log2(X)))))
print(B)

primes = get_b_primes(B, n)
smooth = check_smoothness(math.ceil(math.sqrt(n)), math.ceil(math.pow(n, 0.4)), n, primes)
print(smooth)

3958


In [ ]:
matrix, labels = build_matrix(smooth, primes)
matrix = Matrix(np.mod(np.transpose(matrix), 2))

In [ ]:
kernel = np.transpose(np.transpose(np.array(matrix.nullspace()).astype(np.float64))[0])*10
print(kernel)

In [ ]:
kernDex = np.where(kernel > 0.5)

In [ ]:
print(kernel[kernDex[0][0]])